# 数据聚合和分组运算

## GroupBy 技术

In [32]:
import pandas as pd
import numpy as np

In [33]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,data1,data2,key1,key2
0,-0.367737,-0.785750,a,one
1,-1.005378,-0.506181,a,two
2,-1.100770,0.596523,b,one
3,-0.266099,1.609093,b,two
4,0.773314,0.206055,a,one


In [34]:
# 按key1进行分组 并计算data1列的平均值
grouped = df['data1'].groupby(df['key1'])
grouped.describe()
grouped.mean()

key1
a   -0.199934
b   -0.683435
Name: data1, dtype: float64

In [35]:
means = df['data1'].groupby([df['key1'],  df['key2']]).mean()
means

key1  key2
a     one     0.202789
      two    -1.005378
b     one    -1.100770
      two    -0.266099
Name: data1, dtype: float64

In [36]:
means.unstack()

key2,one,two
key1,,
a,0.202789,-1.005378
b,-1.100770,-0.266099


In [37]:
# 实际上分组建 可以是任意长度适当的数组
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -1.005378
            2006   -1.100770
Ohio        2005   -0.316918
            2006    0.773314
Name: data1, dtype: float64

In [38]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.199934,-0.361958
b,-0.683435,1.102808


In [39]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.202789 -0.289847
     two  -1.005378 -0.506181
b    one  -1.100770  0.596523
     two  -0.266099  1.609093

In [40]:
# groupby的size方法 返回一个含有分组大小的Series
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

In [41]:
for name, group in df.groupby('key1'):
    print(name) # a b 
    print(group) # 数据片段

a
      data1     data2 key1 key2
0 -0.367737 -0.785750    a  one
1 -1.005378 -0.506181    a  two
4  0.773314  0.206055    a  one
b
      data1     data2 key1 key2
2 -1.100770  0.596523    b  one
3 -0.266099  1.609093    b  two


In [42]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
      data1     data2 key1 key2
0 -0.367737 -0.785750    a  one
4  0.773314  0.206055    a  one
a two
      data1     data2 key1 key2
1 -1.005378 -0.506181    a  two
b one
     data1     data2 key1 key2
2 -1.10077  0.596523    b  one
b two
      data1     data2 key1 key2
3 -0.266099  1.609093    b  two


In [43]:
# 将数据片段组成字典
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,-1.100770,0.596523,b,one
3,-0.266099,1.609093,b,two


In [44]:
# groupby默认是在axis=0上进行分组的 通过设置可以在其它任何轴上进行分组
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [45]:
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -0.367737 -0.785750
 1 -1.005378 -0.506181
 2 -1.100770  0.596523
 3 -0.266099  1.609093
 4  0.773314  0.206055, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### 选取一个或一组列

In [46]:
df.groupby('key1')['data1'].mean() # 返回 pd.DataFrame
df.groupby('key1')[['data2']].mean() # 返回  pd.Series

,data2
key1,
a,-0.361958
b,1.102808


In [47]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.289847
     two  -0.506181
b    one   0.596523
     two   1.609093

In [48]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one    -0.289847
      two    -0.506181
b     one     0.596523
      two     1.609093
Name: data2, dtype: float64

### 通过字典或Series进行分组

In [49]:
people = pd.DataFrame(np.random.randn(5, 5), 
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.loc[2:3, ['b', 'c']] = np.nan
people

,a,b,c,d,e
Joe,0.949202,-0.429914,-1.535638,-2.746836,0.694138
Steve,-0.480290,-0.259781,1.693412,-0.080336,-0.437163
Wes,-1.762129,NaN,NaN,1.365047,1.504548
Jim,-1.116987,0.036425,0.073019,0.663599,0.304295
Travis,-0.818263,-0.358654,-0.747208,0.123696,-0.368207


In [50]:
# 已知列的分组关系 根据分组计算列的总计 
# 将字典传给groupby
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}

people.groupby(mapping, axis=1).sum()

,blue,red
Joe,-4.282474,1.213426
Steve,1.613077,-1.177234
Wes,1.365047,-0.257581
Jim,0.736617,-0.776267
Travis,-0.623512,-1.545124


In [51]:
# pd.Series 同样具有一样的功能
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组

In [52]:
people.groupby(len).sum() # fuction len groupby index string length 

,a,b,c,d,e
3,-1.929914,-0.393489,-1.462619,-0.718190,2.502982
5,-0.480290,-0.259781,1.693412,-0.080336,-0.437163
6,-0.818263,-0.358654,-0.747208,0.123696,-0.368207


In [53]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.762129 -0.429914 -1.535638 -2.746836  0.694138
  two -1.116987  0.036425  0.073019  0.663599  0.304295
5 one -0.480290 -0.259781  1.693412 -0.080336 -0.437163
6 two -0.818263 -0.358654 -0.747208  0.123696 -0.368207

### 根据索引级别分组

In [54]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.520623  0.459856 -1.694386  0.897960 -0.416271
1     -0.136694  1.027060 -0.425383 -0.352807 -1.487091
2      1.049265  2.422101 -0.346765  1.085769 -0.389610
3     -0.886952  0.782898  0.432772  1.543552 -0.009654

In [55]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合
### 任何能从数组中产生标量值的数据转化过程

In [56]:
df 
df.groupby('key1')['data1']

In [57]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.545104
b   -0.349566
Name: data1, dtype: float64

In [58]:
# 使用自己的聚合函数 传入agg method
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.778693,0.991805
b,0.834671,1.012570


In [59]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.199934  0.901141 -1.005378 -0.686558 -0.367737  0.202789   
b      2.0 -0.683435  0.590202 -1.100770 -0.892103 -0.683435 -0.474767   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.773314   3.0 -0.361958  0.511390 -0.785750 -0.645965 -0.506181   
b    -0.266099   2.0  1.102808  0.715995  0.596523  0.849666  1.102808   

                          
           75%       max  
key1                      
a    -0.150063  0.206055  
b     1.355951  1.609093

### 面向列的多函数应用

In [60]:
tips = pd.read_csv(r'C:\Users\Administrator\Desktop\计算机入门学习\支线：Python for Data Analysis\pydata-book-2nd-edition\pydata-book-2nd-edition\examples\tips.csv', encoding='utf-8', engine='python')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [61]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean') # 均值

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [62]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [63]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [64]:
function = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(function)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [65]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [66]:
# 传入自定义名称的元组列表
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [67]:
# 传入字典
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [68]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 无索引的形式返回聚合函数

In [69]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 分组级运算和转化 
#### transform 和 apply 方法

In [70]:
df

,data1,data2,key1,key2
0,-0.367737,-0.785750,a,one
1,-1.005378,-0.506181,a,two
2,-1.100770,0.596523,b,one
3,-0.266099,1.609093,b,two
4,0.773314,0.206055,a,one


In [71]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,-0.199934,-0.361958
b,-0.683435,1.102808


In [72]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,-0.367737,-0.785750,a,one,-0.199934,-0.361958
1,-1.005378,-0.506181,a,two,-0.199934,-0.361958
4,0.773314,0.206055,a,one,-0.199934,-0.361958
2,-1.100770,0.596523,b,one,-0.683435,1.102808
3,-0.266099,1.609093,b,two,-0.683435,1.102808


In [73]:
people

,a,b,c,d,e
Joe,0.949202,-0.429914,-1.535638,-2.746836,0.694138
Steve,-0.480290,-0.259781,1.693412,-0.080336,-0.437163
Wes,-1.762129,NaN,NaN,1.365047,1.504548
Jim,-1.116987,0.036425,0.073019,0.663599,0.304295
Travis,-0.818263,-0.358654,-0.747208,0.123696,-0.368207


In [74]:
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

,a,b,c,d,e
one,-0.543730,-0.394284,-1.141423,-0.419365,0.610160
two,-0.798639,-0.111678,0.883215,0.291632,-0.066434


In [75]:
# transform()利用np.mean 依照key进行分组计算 但是按照原index显示
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,-0.543730,-0.394284,-1.141423,-0.419365,0.610160
Steve,-0.798639,-0.111678,0.883215,0.291632,-0.066434
Wes,-0.543730,-0.394284,-1.141423,-0.419365,0.610160
Jim,-0.798639,-0.111678,0.883215,0.291632,-0.066434
Travis,-0.543730,-0.394284,-1.141423,-0.419365,0.610160


In [76]:
# transform会将一个函数应用到各个分组 然后将结果放置到适当的位置上
# 如果各分组产生的是一个标量值 则该值就会被广播出去
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(key).transform(demean)
demeaned # 矩平化函数

,a,b,c,d,e
Joe,1.492932,-0.035630,-0.394215,-2.327471,0.083978
Steve,0.318348,-0.148103,0.810197,-0.371967,-0.370729
Wes,-1.218399,NaN,NaN,1.784411,0.894389
Jim,-0.318348,0.148103,-0.810197,0.371967,0.370729
Travis,-0.274533,0.035630,0.394215,0.543060,-0.978367


In [77]:
demeaned.groupby(key).mean() # 检查现在分组平均值是否为0

,a,b,c,d,e
one,0.0,0.0,5.551115e-17,3.700743e-17,1.110223e-16
two,0.0,0.0,5.551115e-17,2.775558e-17,0.000000e+00


### apply: 一般性的“拆分-应用-合并”

####  跟aggregate一样 transform也是一个有着严格条件的特殊函数：传入的函数只能产生两种结果 要么产生一个可以广播的标量值 要么产生一个相同大小的结果数组

In [78]:
df

,data1,data2,key1,key2
0,-0.367737,-0.785750,a,one
1,-1.005378,-0.506181,a,two
2,-1.100770,0.596523,b,one
3,-0.266099,1.609093,b,two
4,0.773314,0.206055,a,one


In [79]:
df.groupby('key2')['data1'].agg('mean')
df.groupby('key2')['data1'].mean()
df.groupby('key2')['data1'].transform(np.mean)

0   -0.231731
1   -0.635739
2   -0.231731
3   -0.635739
4   -0.231731
Name: data1, dtype: float64

#### apply 会将待处理的对象拆分为多个片段 然后将各片段调用传入的函数 最后尝试将各个片段组合到一起

In [80]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [81]:
def top(df, n=5, column='tip_pct'):
    """小费数据集 根据分组选出最高的5个tip_pct值"""
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [82]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [83]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [84]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [85]:
result.stack().unstack(0)

smoker,No,Yes
count,151.000000,93.000000
mean,0.159328,0.163196
std,0.039910,0.085119
min,0.056797,0.035638
25%,0.136906,0.106771
50%,0.155625,0.153846
75%,0.185014,0.195059
max,0.291990,0.710345


In [86]:
f = lambda x: x.describe()
df.groupby('key1').apply(f)
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.199934  0.901141 -1.005378 -0.686558 -0.367737  0.202789   
b      2.0 -0.683435  0.590202 -1.100770 -0.892103 -0.683435 -0.474767   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.773314   3.0 -0.361958  0.511390 -0.785750 -0.645965 -0.506181   
b    -0.266099   2.0  1.102808  0.715995  0.596523  0.849666  1.102808   

                          
           75%       max  
key1                      
a    -0.150063  0.206055  
b     1.355951  1.609093

### 禁止分组建

In [87]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [88]:
tips.groupby('smoker', group_keys=False).apply(top) # smoker列屏蔽

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析

 #### pandas 能根据指定面元或样本分位数将数据拆分为多块的工具 --cut qcut
 #### 数据集的桶-bucket 或 分位数quantile

In [89]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame.count()

data1    1000
data2    1000
dtype: int64

In [90]:
factor = pd.cut(frame.data1, 4)
factor[:10]

0     (-1.26, 0.275]
1     (-1.26, 0.275]
2     (-1.26, 0.275]
3     (-1.26, 0.275]
4      (0.275, 1.81]
5      (0.275, 1.81]
6      (0.275, 1.81]
7    (-2.801, -1.26]
8     (-1.26, 0.275]
9      (0.275, 1.81]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.801, -1.26] < (-1.26, 0.275] < (0.275, 1.81] < (1.81, 3.346]]

In [91]:
# 由cut返回的factor对象可直接用于groupby
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'conut': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(factor) # = frame.groupby(pd.cut(frame.data1, 4))['data2']

In [92]:
grouped.apply(get_stats).unstack(1)

,conut,max,mean,min
data1,,,,
"(-2.801, -1.26]",113.0,2.897485,-0.128880,-2.823335
"(-1.26, 0.275]",500.0,3.223471,0.094423,-2.779753
"(0.275, 1.81]",358.0,2.901266,-0.042090,-3.879500
"(1.81, 3.346]",29.0,1.887209,0.091024,-1.989547


In [93]:
# 根据样本分位数得到大小相同的桶 使用qcut 传入labels=False即可只获取分位数编号
grouping = pd.qcut(frame.data1, 100, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack().head()

,conut,max,mean,min
data1,,,,
0,10.0,2.829401,0.062820,-1.528987
1,10.0,2.897485,0.056963,-1.268795
2,10.0,2.723173,-0.159348,-1.698219
3,10.0,2.078863,-0.294957,-2.078322
4,10.0,0.485251,-0.090181,-1.429477


### 示例：用特定于分组的值填充缺失值

#### 对于缺失数据的清理工作 dropna将其剔除 fillna进行填充

In [94]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.733320
2         NaN
3   -0.565431
4         NaN
5   -0.641309
dtype: float64

In [95]:
s.fillna(s.mean())

0   -0.157807
1    0.733320
2   -0.157807
3   -0.565431
4   -0.157807
5   -0.641309
dtype: float64

In [96]:
# 对于不同的分组填充不同的值
states = ['Ohio', 'New York', 'Vermont', 'Florida', 
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4

In [97]:
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [98]:
data

Ohio         -1.518093
New York     -1.898866
Vermont            NaN
Florida       0.178003
Oregon       -0.212532
Nevada             NaN
California   -1.332763
Idaho              NaN
dtype: float64

In [99]:
data.groupby(group_key).mean()

East   -1.079652
West   -0.772647
dtype: float64

In [100]:
# 使用分组的平均值进行填充
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -1.518093
New York     -1.898866
Vermont      -1.079652
Florida       0.178003
Oregon       -0.212532
Nevada       -0.772647
California   -1.332763
Idaho        -0.772647
dtype: float64

In [101]:
# 同样也可以在代码中预定义各组的填充值
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.518093
New York     -1.898866
Vermont       0.500000
Florida       0.178003
Oregon       -0.212532
Nevada       -1.000000
California   -1.332763
Idaho        -1.000000
dtype: float64

### 示例：随机采用和排序
#### 从一个大数据集中随机抽取样本以进行蒙特卡罗模拟
#### np.random.permutation(N)的前K个元素 N为完整数据的大小 K作为期望的样本大小

In [102]:
# 构造一幅扑克牌
suits = ['H', 'S', 'C', 'D']
card_val = list(range(1, 11)) + [10] * 3
card_val = card_val * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)


deck = pd.Series(index=cards, data=card_val)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [103]:
# 从整幅牌中抽出5张
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

9C     9
3D     3
3C     3
QH    10
6H     6
dtype: int64

In [104]:
# 假设从每种花色中抽取2张
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  3C      3
   JC     10
D  2D      2
   4D      4
H  5H      5
   AH      1
S  10S    10
   2S      2
dtype: int64

In [105]:
# 另一种办法
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC     10
2C      2
10D    10
AD      1
7H      7
KH     10
6S      6
3S      3
dtype: int64

### 示例：分组加权平均数和相关系数

In [106]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-1.078477,0.796279
1,a,1.536948,0.105664
2,a,-0.521289,0.233429
3,a,2.192815,0.831648
4,b,-1.020807,0.101121
5,b,0.329266,0.778187
6,b,-0.447890,0.994094
7,b,-0.197656,0.376127


In [107]:
df.groupby('category')
get_wag = lambda g: np.average(g['data'], weights=g['weights'])
df.groupby('category').apply(get_wag)

category
a    0.511229
b   -0.162960
dtype: float64

In [108]:
close_px = pd.read_csv(r'C:\Users\Administrator\Desktop\计算机入门学习\支线：Python for Data Analysis\pydata-book-2nd-edition\pydata-book-2nd-edition\examples\stock_px.csv', encoding='utf-8', engine='python', parse_dates=True, index_col=0)
close_px.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,4.98,7.86,2.87,16.79,4.27,0.51,6.04,328.79,6.12
1990-02-02,5.04,8.00,2.87,16.89,4.37,0.51,6.09,330.92,6.24
1990-02-05,5.07,8.18,2.87,17.32,4.34,0.51,6.05,331.85,6.25
1990-02-06,5.01,8.12,2.88,17.56,4.32,0.51,6.15,329.66,6.23
1990-02-07,5.04,7.77,2.91,17.93,4.38,0.51,6.17,333.75,6.33


In [109]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5472 entries, 1990-02-01 to 2011-10-14
Data columns (total 9 columns):
AA      5472 non-null float64
AAPL    5472 non-null float64
GE      5472 non-null float64
IBM     5472 non-null float64
JNJ     5472 non-null float64
MSFT    5472 non-null float64
PEP     5471 non-null float64
SPX     5472 non-null float64
XOM     5472 non-null float64
dtypes: float64(9)
memory usage: 427.5 KB


In [110]:
# pandas.pct_change(periods=1, fill_method='pad', limit=None, freq=None, **kwargs)
# means: Percent change over given number of periods
rets = close_px.pct_change().dropna()

In [111]:
by_year = rets.groupby(lambda x: x.year)
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year.apply(spx_corr)

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990,0.595024,0.545067,0.752187,0.738361,0.801145,0.586691,0.783168,1.0,0.517586
1991,0.453574,0.365315,0.759607,0.557046,0.646401,0.524225,0.641775,1.0,0.569335
1992,0.398180,0.498732,0.632685,0.262232,0.515740,0.492345,0.473871,1.0,0.318408
1993,0.259069,0.238578,0.447257,0.211269,0.451503,0.425377,0.385089,1.0,0.318952
1994,0.428549,0.268420,0.572996,0.385162,0.372962,0.436585,0.450516,1.0,0.395078
1995,0.291532,0.161829,0.519126,0.416390,0.315733,0.453660,0.413144,1.0,0.368752
1996,0.292344,0.191482,0.750724,0.388497,0.569232,0.564015,0.421477,1.0,0.538736
1997,0.564427,0.211435,0.827512,0.646823,0.703538,0.606171,0.509344,1.0,0.695653
1998,0.533802,0.379883,0.815243,0.623982,0.591988,0.698773,0.494213,1.0,0.369264
1999,0.099033,0.425584,0.710928,0.486167,0.517061,0.631315,0.336593,1.0,0.315383


In [112]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

1990    0.408271
1991    0.266807
1992    0.450592
1993    0.236917
1994    0.361638
1995    0.258642
1996    0.147539
1997    0.196144
1998    0.364106
1999    0.329484
2000    0.275298
2001    0.563156
2002    0.571095
2003    0.486262
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 面向分组的线性回归

In [131]:
import statsmodels.api as sm

In [132]:
# statsmodels OLS线性回归 
nsample = 100
x = np.linspace(0, 10, nsample)
X = sm.add_constant(x)
beta = np.array([1, 10])
e = np.random.normal(size=nsample)
beta # 设置模型里的 beta0 beta1

array([ 1, 10])

In [133]:
X # 截据 和x值    y = 1 + 10x

array([[  1.        ,   0.        ],
       [  1.        ,   0.1010101 ],
       [  1.        ,   0.2020202 ],
       [  1.        ,   0.3030303 ],
       [  1.        ,   0.4040404 ],
       [  1.        ,   0.50505051],
       [  1.        ,   0.60606061],
       [  1.        ,   0.70707071],
       [  1.        ,   0.80808081],
       [  1.        ,   0.90909091],
       [  1.        ,   1.01010101],
       [  1.        ,   1.11111111],
       [  1.        ,   1.21212121],
       [  1.        ,   1.31313131],
       [  1.        ,   1.41414141],
       [  1.        ,   1.51515152],
       [  1.        ,   1.61616162],
       [  1.        ,   1.71717172],
       [  1.        ,   1.81818182],
       [  1.        ,   1.91919192],
       [  1.        ,   2.02020202],
       [  1.        ,   2.12121212],
       [  1.        ,   2.22222222],
       [  1.        ,   2.32323232],
       [  1.        ,   2.42424242],
       [  1.        ,   2.52525253],
       [  1.        ,   2.62626263],
 

In [134]:
y = np.dot(X, beta) + e # 反应变量
model = sm.OLS(y,X) # 反应变量 和 回归变量
result = model.fit() # 获取拟合结果
result.params # 计算出的回归系数

array([ 0.88834618,  9.99999344])

In [137]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX']) # 按年计算APPL对spx收益率的线性回归

,SPX,intercept
1990,1.512772,0.001395
1991,1.187351,0.000396
1992,1.832427,0.000164
1993,1.390470,-0.002657
1994,1.190277,0.001617
1995,0.858818,-0.001423
1996,0.829389,-0.001791
1997,0.749928,-0.001901
1998,1.164582,0.004075
1999,1.384989,0.003273


## 透视表和交叉表
### Pandas中的groupby功能以及层次化索引（重塑）运算制作透视表
### DataFrame中有一个pivot_table方法 同样用于构造透视表 还可以添加分项小计

In [146]:
tips.pivot_table(index=['smoker']) # 设置行索引

,size,tip,tip_pct,total_bill
smoker,,,,
No,2.668874,2.991854,0.159328,19.188278
Yes,2.408602,3.008710,0.163196,20.756344


In [148]:
tips.pivot_table(['tip_pct', 'size'], columns='smoker', index='day')

size             tip_pct          
smoker        No       Yes        No       Yes
day                                           
Fri     2.250000  2.066667  0.151650  0.174783
Sat     2.555556  2.476190  0.158048  0.147906
Sun     2.929825  2.578947  0.160113  0.187250
Thur    2.488889  2.352941  0.160298  0.163863

In [151]:
tips.pivot_table(['tip_pct', 'size'], columns='smoker', index='day', margins=True)
# margin=True添加分项小计

size                       tip_pct                    
smoker        No       Yes       All        No       Yes       All
day                                                               
Fri     2.250000  2.066667  2.105263  0.151650  0.174783  0.169913
Sat     2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
Sun     2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
Thur    2.488889  2.352941  2.451613  0.160298  0.163863  0.161276
All     2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [155]:
# 使用聚合函数 传给aggfunc 默认为mean 任何对groupby有效的函数
tips.pivot_table('tip_pct', index='smoker', columns='day',
                  aggfunc='count', margins=True)

day,Fri,Sat,Sun,Thur,All
smoker,,,,,
No,4.0,45.0,57.0,45.0,151.0
Yes,15.0,42.0,19.0,17.0,93.0
All,19.0,87.0,76.0,62.0,244.0


In [157]:
tips.pivot_table('tip_pct', index='smoker', columns='day') # 

day,Fri,Sat,Sun,Thur
smoker,,,,
No,0.151650,0.158048,0.160113,0.160298
Yes,0.174783,0.147906,0.187250,0.163863


In [161]:
m = tips['day'] == 'Sun'
m.sum()

76

In [163]:
tips.pivot_table('size', index=['time', 'smoker'], columns='day',
                 aggfunc='sum', fill_value=0) # fill_value 填充缺失值

day            Fri  Sat  Sun  Thur
time   smoker                     
Dinner No        6  115  167     2
       Yes      20  104   49     0
Lunch  No        3    0    0   110
       Yes      11    0    0    40

### 交叉表：crosstab
#### 是一种用于计算分组频率的特殊透视表

In [165]:
pd.crosstab(index=[tips.time, tips.day], columns=tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244